# testAPI_propane

Created by Davy Yue 2017-06-22


### Imports

In [1]:
import itertools
import string
import os

import numpy as np

from msibi import MSIBI, State, Pair, mie
import mdtraj as md

## PROPANE - edition (code to be expanded)

Where the real magic happens

In [2]:
t = md.load('traj_unwrapped.dcd', top='start_aa.hoomdxml')

# print info for debugging
# print(t)
# print("Number of Chains: {0}".format(t.n_chains))
# print("Topology: {0}".format(t.top))

### Mapping and application

Keys being CG bead indices and values being a list of atom indices corresponding to each CG bead

e.g., {prop0: [0, 1, 2], prop1: [3, 4, 5], prop2: [6, 7, 8], …}

Construct for entire system

In [3]:
cg_idx = 0
start_idx = 0
n_propane = 1024 #passed later
propane_map = {0: [0, 1, 2]}
system_mapping = {}
for n in range(n_propane):
    for bead, atoms in propane_map.items():
        system_mapping[cg_idx] = [x + start_idx for x in atoms]
        start_idx += len(atoms)
        cg_idx += 1
        
        
print(system_mapping)

{0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8], 3: [9, 10, 11], 4: [12, 13, 14], 5: [15, 16, 17], 6: [18, 19, 20], 7: [21, 22, 23], 8: [24, 25, 26], 9: [27, 28, 29], 10: [30, 31, 32], 11: [33, 34, 35], 12: [36, 37, 38], 13: [39, 40, 41], 14: [42, 43, 44], 15: [45, 46, 47], 16: [48, 49, 50], 17: [51, 52, 53], 18: [54, 55, 56], 19: [57, 58, 59], 20: [60, 61, 62], 21: [63, 64, 65], 22: [66, 67, 68], 23: [69, 70, 71], 24: [72, 73, 74], 25: [75, 76, 77], 26: [78, 79, 80], 27: [81, 82, 83], 28: [84, 85, 86], 29: [87, 88, 89], 30: [90, 91, 92], 31: [93, 94, 95], 32: [96, 97, 98], 33: [99, 100, 101], 34: [102, 103, 104], 35: [105, 106, 107], 36: [108, 109, 110], 37: [111, 112, 113], 38: [114, 115, 116], 39: [117, 118, 119], 40: [120, 121, 122], 41: [123, 124, 125], 42: [126, 127, 128], 43: [129, 130, 131], 44: [132, 133, 134], 45: [135, 136, 137], 46: [138, 139, 140], 47: [141, 142, 143], 48: [144, 145, 146], 49: [147, 148, 149], 50: [150, 151, 152], 51: [153, 154, 155], 52: [156, 157, 158], 53: [1

With mapping for whole system, apply to all atom trajectory

In [4]:
from mdtraj.core import element
list(t.top.atoms)[0].element = element.carbon
list(t.top.atoms)[0].element.mass
for atom in t.top.atoms:
    atom.element = element.carbon

In [5]:
cg_xyz = np.empty((t.n_frames, len(system_mapping), 3))
for cg_bead, aa_indices in system_mapping.items():
    cg_xyz[:, cg_bead, :] = md.compute_center_of_mass(t.atom_slice(aa_indices))
    
print(cg_xyz)

[[[-0.49629103  0.82075842 -0.04100191]
  [-0.74373281 -0.95105817  1.02634521]
  [-0.154862   -0.36611393 -0.22066766]
  ..., 
  [ 0.59285714  0.41730833 -0.18561413]
  [-0.62547243  0.78334932 -0.56067938]
  [ 0.73455014  0.76501046  0.51811039]]

 [[-1.1597714  -0.14085296 -1.00486334]
  [-0.42053872  1.10189251 -0.77430379]
  [ 0.65535567 -0.2891442   0.74442508]
  ..., 
  [ 0.52092607  1.03908714 -0.64020044]
  [ 0.68245687  1.21493439 -0.49101884]
  [-0.51417306 -0.92634368  0.64851546]]

 [[-0.65758572 -0.78551328  0.20813531]
  [-0.68148766 -1.007828   -1.12453334]
  [ 0.87474823  1.16547847 -0.77278932]
  ..., 
  [-0.05327949 -0.29133598 -0.6095888 ]
  [ 0.8360339   0.26535088 -1.02305935]
  [-1.14497372  0.96994674  0.33847151]]

 ..., 
 [[-1.20415286  0.04921983  0.90470382]
  [-0.98199191 -0.47386939 -0.94287425]
  [ 0.95282167 -1.03314833  0.49292536]
  ..., 
  [-0.95389412  0.98665885 -1.27135094]
  [ 0.41141723 -0.8066449  -1.28323976]
  [-1.11254172 -0.354603   -1.27716

### Traj & Obj

* Create new Trajectory object & CG Topology object

* Save resultant trajectory file

In [44]:
# print(atom.element)
# print(t.top.residues)
# print(t)

cg_top = md.Topology()
topology = md.Topology()
for cg_bead in system_mapping.keys():
    #add_atom(name, element, residue, serial=None)
    cg_top.add_atom('carbon',atom.element, topology.add_residue('A', topology.add_chain())) 
    # add_atom('name', element, topology.add_residue('A', topology.add_chain()))
    
# class mdtraj.Trajectory(xyz, topology, time=None, unitcell_lengths=None, unitcell_angles=None)
cg_traj = md.Trajectory(cg_xyz, cg_top, time=None, unitcell_lengths=t.unitcell_lengths, unitcell_angles=t.unitcell_angles)
cg_traj.save_dcd('cg_traj.dcd')

In [47]:
print(cg_traj)
print(cg_top)
print(cg_xyz)

<mdtraj.Trajectory with 500 frames, 1024 atoms, 0 residues, and unitcells>
<mdtraj.Topology with 0 chains, 0 residues, 1024 atoms, 0 bonds>
[[[-0.49629103  0.82075842 -0.04100191]
  [-0.74373281 -0.95105817  1.02634521]
  [-0.154862   -0.36611393 -0.22066766]
  ..., 
  [ 0.59285714  0.41730833 -0.18561413]
  [-0.62547243  0.78334932 -0.56067938]
  [ 0.73455014  0.76501046  0.51811039]]

 [[-1.1597714  -0.14085296 -1.00486334]
  [-0.42053872  1.10189251 -0.77430379]
  [ 0.65535567 -0.2891442   0.74442508]
  ..., 
  [ 0.52092607  1.03908714 -0.64020044]
  [ 0.68245687  1.21493439 -0.49101884]
  [-0.51417306 -0.92634368  0.64851546]]

 [[-0.65758572 -0.78551328  0.20813531]
  [-0.68148766 -1.007828   -1.12453334]
  [ 0.87474823  1.16547847 -0.77278932]
  ..., 
  [-0.05327949 -0.29133598 -0.6095888 ]
  [ 0.8360339   0.26535088 -1.02305935]
  [-1.14497372  0.96994674  0.33847151]]

 ..., 
 [[-1.20415286  0.04921983  0.90470382]
  [-0.98199191 -0.47386939 -0.94287425]
  [ 0.95282167 -1.03314

### Calculate RDF and save

In [55]:
pairs = cg_traj.top.select_pairs(selection1='name A', selection2='name A')
print(pairs)
# mdtraj.compute_rdf(traj, pairs=None, r_range=None, bin_width=0.005, n_bins=None, periodic=True, opt=True)
np.savetxt('rdfs_aa.txt', np.transpose(md.compute_rdf(cg_traj, pairs=pairs)))
# https://stackoverflow.com/questions/15192847/saving-arrays-as-columns-with-np-savetxt

[]


/Users/davyyue/anaconda3/envs/msibi/lib/python3.5/site-packages/mdtraj/geometry/rdf.py:92: RuntimeWarning: invalid value encountered in true_divide
  g_r = g_r.astype(np.float64) / norm  # From int64.
